In [6]:
# tools: reducing the number of nets to one
import numpy as np

def simplify_board(filename):
    board = np.loadtxt(filename, delimiter=',')
    for i in range(board.shape[0]):
        for j in range(board.shape[1]):
            if abs(board[i,j])>2:
                board[i,j] = 1
    np.savetxt("sim_board.csv", board, delimiter=',')
    return board
board = simplify_board("testboard.csv")
print(np.amin(board))

-2.0


In [ ]:
# test gossiping routing
import numpy as np
import random
from copy import copy

def cos_value(B, A, D):
    '''
    B, A, D should be numpy arrays
    return cos(BAD)
    '''
    AB_vec = B-A
    AD_vec = D-A
    dot_product = np.dot(AB_vec, AD_vec)
    AB = np.linalg.norm(AB_vec)
    AD = np.linalg.norm(AD_vec)
    return dot_product/(AB*AD)

def Score1(B, A, D):
    return cos_value(B,A,D)

def Score2(B,A,D):
    return 1 + cos_value(B,A,D)

def Score3(B,A,D):
    e = 0.3
    return (1-e)/(1-e*cos_value(B,A,D))

def find_node(board, number):
    '''
    return a numpy array with 2 elements
    '''
    return np.asarray(np.where(board == number)).reshape(-1)

def rollout(board):
    boards = [board]
#     current_node = find_node(board)
    paths = [[]]
    final_paths = []
    while len(final_paths)<20:
        boards_tem = []
        paths_tem = []
        for i in range(len(boards)):
            b = boards[i]
            p = paths[i]
            bs_tem, ps_tem, path = take_actions(b,p)
            if path is not None:
                final_paths.append(path)
            boards_tem += bs_tem
            paths_tem += ps_tem
        boards = boards_tem
        paths = paths_tem
    return final_paths

def take_actions(board, path):
    '''
    return a list of 2d numpy array (board)
    '''
    net_number = 2
    boards_next = []
    paths_next = []
    current_node = find_node(board, net_number)
    neighbors = find_neighbors(board, current_node)
    
    if len(neighbors)==0:
        return [], [], None
    
    destination = find_node(board, -net_number)
    probs = calculate_probs(neighbors, current_node, destination)
    
    if tuple(destination) in neighbors:
        return boards_next, paths_next, path
        
    for neighbor in neighbors:
        prob = probs[neighbor]
        if random.random()<=prob:
            board_next = copy(board)
            path_next = copy(path)
            board_next[neighbor] = net_number
            board_next[tuple(current_node)] = 1
            path_next.append(neighbor)
            boards_next.append(board_next)
            paths_next.append(path_next)
            
    return boards_next, paths_next, None

def find_neighbors(board, current_node):
    '''
    current_node is a numpy array
    return:
    neighbors a list of tuples
    '''
    neighbors = []
    directions = np.array([[0,1], [0,-1], [1,0], [-1,0]])
    for d in directions:
        candidate_neighbor = tuple(current_node + d)
        if np.amax(candidate_neighbor)<board.shape[0] and np.amin(candidate_neighbor)>=0:
            if board[candidate_neighbor]!=1:
                neighbors.append(candidate_neighbor)
    return neighbors

def calculate_probs(neighbors, current_node, destination):
    '''
    return a dict mapping the tuple-based neighbors to probs 
    '''
    scores = []
    probs = dict()
    for neighbor in neighbors:
        scores.append(Score3(np.asarray(neighbor), current_node, destination))
        
    max_score = max(scores)
    for neighbor in neighbors:
        p = Score3(np.asarray(neighbor), current_node, destination)/max_score
        probs[neighbor] = p
    return probs

board = np.loadtxt("sim_board.csv", delimiter=',')
routes = rollout(board)


[[(7, 13)], [(6, 12)]]
[[(7, 13), (7, 14)], [(7, 13), (8, 13)], [(6, 12), (6, 13)], [(6, 12), (6, 11)]]
[[(7, 13), (7, 14), (7, 15)], [(7, 13), (7, 14), (8, 14)], [(7, 13), (7, 14), (6, 14)], [(7, 13), (8, 13), (8, 14)], [(6, 12), (6, 13), (6, 14)], [(6, 12), (6, 13), (7, 13)], [(6, 12), (6, 13), (5, 13)], [(6, 12), (6, 11), (7, 11)], [(6, 12), (6, 11), (5, 11)]]
[[(7, 13), (7, 14), (7, 15), (7, 16)], [(7, 13), (7, 14), (7, 15), (8, 15)], [(7, 13), (7, 14), (7, 15), (6, 15)], [(7, 13), (7, 14), (8, 14), (8, 15)], [(7, 13), (7, 14), (8, 14), (8, 13)], [(7, 13), (7, 14), (8, 14), (9, 14)], [(7, 13), (7, 14), (6, 14), (6, 15)], [(7, 13), (8, 13), (8, 14), (8, 15)], [(7, 13), (8, 13), (8, 14), (9, 14)], [(6, 12), (6, 13), (6, 14), (6, 15)], [(6, 12), (6, 13), (7, 13), (7, 14)], [(6, 12), (6, 13), (7, 13), (8, 13)], [(6, 12), (6, 13), (5, 13), (5, 14)], [(6, 12), (6, 13), (5, 13), (4, 13)], [(6, 12), (6, 11), (7, 11), (7, 10)], [(6, 12), (6, 11), (5, 11), (5, 12)], [(6, 12), (6, 11), (5, 11

[[(7, 13), (7, 14), (7, 15), (7, 16), (7, 17), (7, 18), (7, 19), (7, 20), (7, 21), (7, 22)], [(7, 13), (7, 14), (7, 15), (7, 16), (7, 17), (7, 18), (7, 19), (7, 20), (7, 21), (8, 21)], [(7, 13), (7, 14), (7, 15), (7, 16), (7, 17), (7, 18), (7, 19), (8, 19), (9, 19), (9, 20)], [(7, 13), (7, 14), (7, 15), (7, 16), (7, 17), (7, 18), (7, 19), (8, 19), (9, 19), (9, 18)], [(7, 13), (7, 14), (7, 15), (7, 16), (7, 17), (7, 18), (7, 19), (8, 19), (9, 19), (10, 19)], [(7, 13), (7, 14), (7, 15), (7, 16), (7, 17), (7, 18), (7, 19), (6, 19), (6, 20), (6, 21)], [(7, 13), (7, 14), (7, 15), (7, 16), (7, 17), (7, 18), (7, 19), (6, 19), (6, 20), (7, 20)], [(7, 13), (7, 14), (7, 15), (7, 16), (7, 17), (7, 18), (7, 19), (6, 19), (6, 20), (5, 20)], [(7, 13), (7, 14), (7, 15), (7, 16), (7, 17), (7, 18), (7, 19), (6, 19), (5, 19), (5, 20)], [(7, 13), (7, 14), (7, 15), (7, 16), (7, 17), (7, 18), (7, 19), (6, 19), (5, 19), (4, 19)], [(7, 13), (7, 14), (7, 15), (7, 16), (7, 17), (7, 18), (6, 18), (6, 19), (6, 2

[[(7, 13), (7, 14), (7, 15), (7, 16), (7, 17), (7, 18), (7, 19), (7, 20), (7, 21), (7, 22), (7, 23)], [(7, 13), (7, 14), (7, 15), (7, 16), (7, 17), (7, 18), (7, 19), (7, 20), (7, 21), (7, 22), (8, 22)], [(7, 13), (7, 14), (7, 15), (7, 16), (7, 17), (7, 18), (7, 19), (7, 20), (7, 21), (7, 22), (6, 22)], [(7, 13), (7, 14), (7, 15), (7, 16), (7, 17), (7, 18), (7, 19), (7, 20), (7, 21), (8, 21), (8, 22)], [(7, 13), (7, 14), (7, 15), (7, 16), (7, 17), (7, 18), (7, 19), (7, 20), (7, 21), (8, 21), (9, 21)], [(7, 13), (7, 14), (7, 15), (7, 16), (7, 17), (7, 18), (7, 19), (8, 19), (9, 19), (9, 20), (9, 21)], [(7, 13), (7, 14), (7, 15), (7, 16), (7, 17), (7, 18), (7, 19), (8, 19), (9, 19), (9, 20), (10, 20)], [(7, 13), (7, 14), (7, 15), (7, 16), (7, 17), (7, 18), (7, 19), (8, 19), (9, 19), (9, 18), (10, 18)], [(7, 13), (7, 14), (7, 15), (7, 16), (7, 17), (7, 18), (7, 19), (8, 19), (9, 19), (9, 18), (8, 18)], [(7, 13), (7, 14), (7, 15), (7, 16), (7, 17), (7, 18), (7, 19), (8, 19), (9, 19), (10, 1

[[(7, 13), (7, 14), (7, 15), (7, 16), (7, 17), (7, 18), (7, 19), (7, 20), (7, 21), (7, 22), (7, 23), (7, 24)], [(7, 13), (7, 14), (7, 15), (7, 16), (7, 17), (7, 18), (7, 19), (7, 20), (7, 21), (7, 22), (7, 23), (8, 23)], [(7, 13), (7, 14), (7, 15), (7, 16), (7, 17), (7, 18), (7, 19), (7, 20), (7, 21), (7, 22), (7, 23), (6, 23)], [(7, 13), (7, 14), (7, 15), (7, 16), (7, 17), (7, 18), (7, 19), (7, 20), (7, 21), (7, 22), (8, 22), (8, 23)], [(7, 13), (7, 14), (7, 15), (7, 16), (7, 17), (7, 18), (7, 19), (7, 20), (7, 21), (7, 22), (8, 22), (8, 21)], [(7, 13), (7, 14), (7, 15), (7, 16), (7, 17), (7, 18), (7, 19), (7, 20), (7, 21), (7, 22), (8, 22), (9, 22)], [(7, 13), (7, 14), (7, 15), (7, 16), (7, 17), (7, 18), (7, 19), (7, 20), (7, 21), (7, 22), (6, 22), (6, 23)], [(7, 13), (7, 14), (7, 15), (7, 16), (7, 17), (7, 18), (7, 19), (7, 20), (7, 21), (7, 22), (6, 22), (5, 22)], [(7, 13), (7, 14), (7, 15), (7, 16), (7, 17), (7, 18), (7, 19), (7, 20), (7, 21), (8, 21), (8, 22), (8, 23)], [(7, 13),

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [17]:
print(len(paths))
a = [print(p) for p in paths]

31
[(7, 13), (7, 14), (7, 15), (7, 16), (7, 17), (7, 18), (7, 19), (7, 20), (7, 21), (7, 22), (7, 23), (7, 24), (7, 25), (7, 26), (7, 27), (8, 27), (8, 26), (9, 26), (9, 27), (10, 27), (10, 28), (10, 29), (10, 30), (10, 31), (10, 32), (9, 32)]
[(7, 13), (7, 14), (7, 15), (7, 16), (7, 17), (7, 18), (7, 19), (7, 20), (7, 21), (7, 22), (7, 23), (7, 24), (7, 25), (7, 26), (7, 27), (8, 27), (9, 27), (9, 26), (10, 26), (10, 27), (10, 28), (10, 29), (10, 30), (10, 31), (10, 32), (9, 32)]
[(7, 13), (7, 14), (7, 15), (7, 16), (7, 17), (7, 18), (7, 19), (7, 20), (7, 21), (7, 22), (7, 23), (7, 24), (7, 25), (7, 26), (7, 27), (8, 27), (9, 27), (9, 26), (10, 26), (10, 27), (11, 27), (11, 28), (11, 29), (11, 30), (10, 30), (10, 31), (10, 32), (9, 32)]
[(7, 13), (7, 14), (7, 15), (7, 16), (7, 17), (7, 18), (7, 19), (7, 20), (7, 21), (7, 22), (7, 23), (7, 24), (7, 25), (7, 26), (7, 27), (8, 27), (9, 27), (9, 26), (10, 26), (10, 27), (11, 27), (11, 28), (11, 29), (10, 29), (10, 30), (10, 31), (10, 32),